In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import glob, pylab, pandas as pd
import pydicom, numpy as np

# Any results you write to the current directory are saved as output.

In [ ]:
#reading the data and splitting it into positive and negative cases!!!
#getting the unique positive cases
df = pd.read_csv('../input/stage_2_train_labels.csv')
print(df.shape)
negative=df[df.Target==0]
positive=df[df.Target==1]
pcases=positive.patientId.unique()
print(negative.shape,positive.shape,pcases.shape)

In [ ]:
pcases[0]

In [ ]:
#reading the ground truth and structuring it from  width-height format to min_max format
min_max_cases=[] 
ratios=[]
sizes=[]
for p in pcases[:10]:
    
    min_max_case=[]
    for i in range(len(positive[positive.patientId==p])):
        min_max=[]
        min_max.append(positive[positive.patientId==p].x.iloc[i])
        min_max.append(positive[positive.patientId==p].y.iloc[i])
        min_max.append(positive[positive.patientId==p].x.iloc[i]+positive[positive.patientId==p].width.iloc[i])
        min_max.append(positive[positive.patientId==p].y.iloc[i]+positive[positive.patientId==p].height.iloc[i])
        min_max_case.append(min_max)
        #ratios.append(positive[positive.patientId==p].height.iloc[i]/positive[positive.patientId==p].width.iloc[i])
        #sizes.append(positive[positive.patientId==p].height.iloc[i]*positive[positive.patientId==p].width.iloc[i])
    min_max_cases.append(min_max_case) 
print(len(min_max_cases))

In [ ]:
(min_max_cases[0])

In [ ]:
#getting the iamge data
img_data=[]
for p in pcases[:10]:
    dcm_file = '../input/stage_2_train_images/%s.dcm' % p
    dcm_data = pydicom.read_file(dcm_file)
    img_data.append(dcm_data.pixel_array)
img_data=np.expand_dims(img_data,axis=-1)
img_data.shape

In [ ]:
def intersection_area (gt,anchors):
    x1_min,y1_min,x1_max,y1_max=gt
    x2_min,y2_min,x2_max,y2_max=anchors
    
    x_min_max=max(x1_min,x2_min)
    y_min_max=max(y1_min,y2_min)
    x_max_min=min(x1_max,x2_max)
    y_max_min=min(y1_max,y2_max)
    if((x_max_min-x_min_max)<0)or((y_max_min-y_min_max)<0):
        return (0)
    else:
        return((x_max_min-x_min_max)*(y_max_min-y_min_max))

def union_area(gt,anchors):
    x1_min,y1_min,x1_max,y1_max=gt
    x2_min,y2_min,x2_max,y2_max=anchors
    
    x_min=min(x1_min,x2_min)
    y_min=min(y1_min,y2_min)
    x_max=max(x1_max,x2_max)
    y_max=max(y1_max,y2_max)
    if((x_max-x_min)<0)or((y_max-y_min)<0):
        return(0)
    else:
        return((x_max-x_min)*(y_max-y_min))

def iou(gt,anchors):
    unionArea=union_area(gt,anchors)
    intersectionArea=intersection_area(gt,anchors)
    #print(unionArea,intersectionArea)
    iou=intersectionArea/(unionArea+0.000000000000000000000000000000000001)
    #print(iou)
    return(iou)
    


In [ ]:
def findRegression(anchor_box_xy,boxes):
    
    r1=((boxes[2]+boxes[0])/2-(anchor_box_xy[2]+anchor_box_xy[0])/2)/(anchor_box_xy[2]+anchor_box_xy[0])
    r2=((boxes[3]+boxes[1])/2-(anchor_box_xy[3]+anchor_box_xy[1])/2)/(anchor_box_xy[3]+anchor_box_xy[1])
    tw=((boxes[2]-boxes[0])/(anchor_box_xy[2]-anchor_box_xy[0]))
    th=((boxes[3]-boxes[1])/(anchor_box_xy[3]-anchor_box_xy[1]))
    return(r1,r2,tw,th)

In [ ]:
findRegression([0,0,100,100],[0,0,100,100])

In [ ]:
anchor_ratios=([[1,1],[1,2],[2,1]])#these things will be decided once the data is explored
anchor_positions=[0,0.25,0.5,0.75]
anchor_ratios=np.array(anchor_ratios)
feature_map_sizes=[32,16,8]#feature map size
input_size=(1024,1024)
noc=2#including bg

In [ ]:
#label_encoding
import math
iou_all=[]
max_iou_value_all=[]
flat_all=[]

classifier_feature_map=[]
anchor_box_feature_map=[]
regressor_feature_map=[]
for count_fms,fms in enumerate(feature_map_sizes):
    classifier_feature_map.append(np.zeros((len(min_max_cases),fms,fms,noc)))
    anchor_box_feature_map.append(np.zeros((len(min_max_cases),fms,fms,
                                            len(anchor_positions),len(anchor_positions),len(anchor_ratios))))
    regressor_feature_map.append(np.zeros((len(min_max_cases),fms,fms,
                                            len(anchor_positions),len(anchor_positions),len(anchor_ratios),4)))
for count_case,case in enumerate(min_max_cases[:10]):
    
    iou_case=[]
    max_iou_value_case=[]
    flat_case=[]
    for count_box, box in enumerate(case):
        target=1#will change for multiple classes
        iou_boxes=[]
        flat_boxes=[]
        max_iou_value_box=[]
        for count_fms,fms in enumerate(feature_map_sizes):
            x_position=np.arange(fms)
            y_position=np.arange(fms)

            x_positions=np.add.outer(x_position,anchor_positions)
            y_positions=np.add.outer(y_position,anchor_positions)

            x_positions_upscaled=(x_positions)*(input_size[0]/fms)
            y_positions_upscaled=(y_positions)*(input_size[0]/fms)
            
            x_left=np.multiply.outer([x_positions_upscaled-(input_size[0]/(2*fms))],(anchor_ratios[:,0]))
            y_up=np.multiply.outer([y_positions_upscaled-(input_size[1]/(2*fms))],anchor_ratios[:,1])
            x_right=np.multiply.outer([x_positions_upscaled+(input_size[0]/(2*fms))],anchor_ratios[:,0])
            y_down=np.multiply.outer([y_positions_upscaled+(input_size[1]/(2*fms))],anchor_ratios[:,1])
            iou_box=[]
            flat_box=[]
            for x in range(fms):
                iou_rows=[]
                for y in range(fms):
                    iou_columns=[]
                    for a in range(len(anchor_positions)):
                        iou_a=[]
                        for b in range(len(anchor_positions)):
                            iou_b=[]
                            for z in range(len(anchor_ratios)):
                                iou_value=(iou([x_left[0][x][a][z],y_up[0][y][b][z],x_right[0][x][a][z],y_down[0][y][b][z]],box))
                                if (iou_value>0.7):
                                    classifier_feature_map[count_fms][count_case][x][y][target]=1
                                    anchor_box_feature_map[count_fms][count_case][x][y][a][b][z]=iou_value
                                    regression_factors=findRegression([x_left[0][x][a][z],y_up[0][y][b][z],x_right[0][x][a][z],y_down[0][y][b][z]],box)
                                    regressor_feature_map[count_fms][count_case][x][y][a][b][z]=regression_factors
#classifier_feature_map                                    

In [ ]:
ffm=[]
rfm=[]
afm=[]
for count_fms,fms in enumerate(feature_map_sizes):
    ffm.append(classifier_feature_map[count_fms].reshape((10,(fms*fms),2)))
    rfm.append(regressor_feature_map[count_fms].reshape((10,(fms*fms),192)))
    afm.append(anchor_box_feature_map[count_fms].reshape(10,(fms*fms),48))
#classifier_feature_map_1=classifier_feature_map
classifier_concatenated=np.concatenate((ffm[0],ffm[1],ffm[2]),axis=1)
regressor_concatenated=np.concatenate((rfm[0],rfm[1],rfm[2]),axis=1)
anchor_concatenated=np.concatenate((afm[0],afm[1],afm[2]),axis=1)
print(classifier_concatenated.shape,regressor_concatenated.shape,anchor_concatenated.shape)

#rfm=[]
#for count_rms, rms in enumerate(regresion_feature_map):

In [ ]:
print(len(classifier_feature_map))
print(anchor_box_feature_map[0].shape,regressor_feature_map[0].shape)

In [ ]:
#creating the model
import keras
from keras.models import Model
from keras.layers import Input,Conv2D, Reshape,Concatenate
from keras.regularizers import l2
l2_reg=0.0005

x= Input(shape=(1024,1024,1))
conv1 = Conv2D(3, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv1')(x)
conv2 = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv2')(conv1)
conv3 = Conv2D(16, (3, 3), activation='relu', padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv3')(conv2)
conv4 = Conv2D(16, (3, 3),strides=2, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv4')(conv3)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv5')(conv4)
conv6 = Conv2D(32, (3, 3),strides=2, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv6')(conv5)
conv7 = Conv2D(64, (3, 3), activation='relu', padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv7')(conv6)
conv8 = Conv2D(64, (3, 3),strides=2, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv8')(conv7)
conv9 = Conv2D(128, (3, 3), activation='relu', padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv9')(conv8)
conv10 = Conv2D(256, (3, 3),strides=2, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv10')(conv9)

conv11 = Conv2D(512, (3, 3),strides=2, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11')(conv10)

conv12 = Conv2D(512, (3, 3),strides=2, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12')(conv11)

conv13 = Conv2D(512, (3, 3),strides=2, activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13')(conv12)

class11a = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11a')(conv11)
class11b = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11b')(class11a)
class11c = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11c')(class11b)
class11d = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11d')(class11c)
class11e = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11e')(class11d)
class11f = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11f')(class11e)
class11g = Conv2D(noc, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11g')(class11f)

class12a = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12a')(conv12)
class12b = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12b')(class12a)
class12c = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12c')(class12b)
class12d = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12d')(class12c)
class12e = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12e')(class12d)
class12f = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12f')(class12e)
class12g = Conv2D(noc, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12g')(class12f)

class13a = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13a')(conv13)
class13b = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13b')(class13a)
class13c = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13c')(class13b)
class13d = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13d')(class13c)
class13e = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13e')(class13d)
class13f = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13f')(class13e)
class13g = Conv2D(noc, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13g')(class13f)

print(class11g.shape)
flattened_class11g=Reshape(((32*32), 2), name='flattened_conv_11')(class11g)
flattened_class12g=Reshape(((16*16), 2), name='flattened_conv_12')(class12g)
flattened_class13g=Reshape(((8*8), 2), name='flattened_conv_13')(class13g)

classifier_concatenated=Concatenate(axis=1,name="classifier_concatenated")([flattened_class11g,flattened_class12g,flattened_class13g])

print(classifier_concatenated.shape)

regr11a = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='regr11a')(conv11)
regr11b = Conv2D(192, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='regr11b')(regr11a)
#conv11C = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11C')(conv11B)
#conv11D = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11D')(conv11C)
#conv11E = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11E')(conv11D)
#conv11F = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11F')(conv11E)
#conv11G = Conv2D(4, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11G')(conv11F)

regr12a = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='regr12a')(conv12)
regr12b = Conv2D(192, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='regr12b')(regr12a)
#conv12C = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12C')(conv12B)
#conv12D = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12D')(conv12C)
#conv12E = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12E')(conv12D)
#conv12F = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12F')(conv12E)
#conv12G = Conv2D(4, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12G')(conv12F)

regr13a = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='regr13a')(conv13)
regr13b = Conv2D(192, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='regr13b')(regr13a)
#conv13C = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13C')(conv13B)
#conv13D = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13D')(conv13C)
#conv13E = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13E')(conv13D)
#conv13F = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13F')(conv13E)
#conv13G = Conv2D(4, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13G')(conv13F)

flattened_regr11b=Reshape(((32*32), 192), name='flattened_conv_11_reg')(regr11b)
flattened_regr12b=Reshape(((16*16), 192), name='flattened_conv_12_reg')(regr12b)
flattened_regr13b=Reshape(((8*8), 192), name='flattened_conv_13_reg')(regr13b)

regressor_concatenated=Concatenate(axis=1,name="regressor_concatenated")([flattened_regr11b,flattened_regr12b,flattened_regr13b])

anchor_11a = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='anchor11a')(conv11)
anchor_11b = Conv2D(192, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='anchor11b')(anchor_11a)
anchor_11c = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='anchor11c')(anchor_11b)
anchor_11d = Conv2D(48, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='anchor11d')(anchor_11c)
#conv11E = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11E')(conv11D)
#conv11F = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11F')(conv11E)
#conv11G = Conv2D(4, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv11G')(conv11F)

anchor_12a = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='anchor12a')(conv12)
anchor_12b = Conv2D(192, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='anchor12b')(anchor_12a)
anchor_12c = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='anchor12c')(anchor_12b)
anchor_12d = Conv2D(48, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='anchor12d')(anchor_12c)
#conv12E = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12E')(conv12D)
#conv12F = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12F')(conv12E)
#conv12G = Conv2D(4, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv12G')(conv12F)

anchor_13a = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='anchor13a')(conv13)
anchor_13b = Conv2D(192, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='anchor13b')(anchor_13a)
anchor_13c = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='anchor13c')(anchor_13b)
anchor_13d = Conv2D(48, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='anchor13d')(anchor_13c)
#conv13E = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13E')(conv13D)
#conv13F = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13F')(conv13E)
#conv13G = Conv2D(4, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv13G')(conv13F)

flattened_anchor_11d=Reshape(((32*32), 48), name='flattened_conv_11_anchor')(anchor_11d)
flattened_anchor_12d=Reshape(((16*16), 48), name='flattened_conv_12_anchor')(anchor_12d)
flattened_anchor_13d=Reshape(((8*8), 48), name='flattened_conv_13_anchor')(anchor_13d)

anchor_concatenated=Concatenate(axis=1,name="anchor_concatenated")([flattened_anchor_11d,flattened_anchor_12d,flattened_anchor_13d])

model = Model(inputs=x, outputs=[classifier_concatenated,regressor_concatenated,anchor_concatenated])
print(x.shape,classifier_concatenated.shape,regressor_concatenated.shape,anchor_concatenated.shape)

In [ ]:
from keras import optimizers
from keras import losses
model.compile(optimizer=optimizers.Adagrad(lr=0.001),loss=["mean_squared_error","mean_squared_error","mean_squared_error"] ,metrics=['acc',"acc","acc"])
print(img_data.shape)
model.fit(x=[img_data],y=[classifier_concatenated,regressor_concatenated,anchor_concatenated],batch_size=2,epochs=2)